In [1]:
import torch
import torchvision.transforms as T

import PIL

import os
import random
import cv2
import numpy as np

# from matplotlib import pyplot as plt 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import copy
import json


/homes/e34960/anaconda3/envs/patchdiff/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
apricot_dataset = "/project/trinity/datasets/apricot/pub/apricot-mask/data_mask_v2"
apricot_files = os.listdir(apricot_dataset)

In [3]:
annotations_file = "/project/trinity/datasets/APRICOT_ORIGINAL/Annotations/apricot_annotations_test.json"

with open(annotations_file, 'r') as file:
    data = json.load(file)

annotations_coco_91 = {}

for i in range(len(data['categories'])):
    annotations_coco_91[data['categories'][i]['id']] = data['categories'][i]['name']

In [4]:
def getRandomImageMasks(n=1, scale=1.0, shape=(512,512)):
    random_numbers = random.sample(range(0, len(apricot_files)-1), n)

    imgs = []
    masks = []
    all_bboxes = []
    all_labels = []
    for i in range(n):
        img_info = torch.load(os.path.join(apricot_dataset, apricot_files[random_numbers[i]]))
        img = np.squeeze(img_info['Image'])
        img = (255.0 * img).astype(np.uint8)
        h, w, _ = img.shape

        # img = cv2.resize(img, (int(w * scale), int(h * scale)))
        img = cv2.resize(img, shape)
        imgs.append(img)

        mask = np.squeeze(img_info['Mask'])
        mask = (255.0 * mask).astype(np.uint8)

        # mask = cv2.resize(mask, (int(w * scale), int(h * scale)))
        mask = cv2.resize(mask, shape)

        # Make a rectangular mask
        nonzero_indices = np.nonzero(mask)
        min_row = np.min(nonzero_indices[0])
        max_row = np.max(nonzero_indices[0])
        min_col = np.min(nonzero_indices[1])
        max_col = np.max(nonzero_indices[1])
        mask[min_row:max_row+1, min_col:max_col+1] = 255

        masks.append(mask)


        bounding_boxes = img_info['Annotations'][0]['boxes']
        bboxes = np.reshape(bounding_boxes, (bounding_boxes.shape[1], bounding_boxes.shape[2]))        
        bboxes[:, 2] = ((bboxes[:, 2] - bboxes[:, 0]) * img.shape[0]).astype(int)
        bboxes[:, 3] = ((bboxes[:, 3] - bboxes[:, 1]) * img.shape[1]).astype(int)
        bboxes[:, 0] = (img.shape[0] * bboxes[:, 0]).astype(int)
        bboxes[:, 1] = (img.shape[1] * bboxes[:, 1]).astype(int)

        all_bboxes.append(bboxes)

        labels = img_info['Annotations'][0]['labels']
        labels = np.reshape(labels, (labels.shape[1]))
        all_labels.append(labels)

    return imgs, masks, all_bboxes, all_labels

In [5]:
pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16)
pipe.to("cuda:2")
prompt = ""

In [7]:
def getRepaintedImages(imgs, masks):
    repainted_images = []
    for i in range(len(imgs)):
        repainted_image = pipe(prompt=prompt, image=imgs[i], mask_image=masks[i]).images[0]
        repainted_images.append(repainted_image)
    
    return repainted_images

In [8]:
num_images = 20
imgs, masks, all_bboxes, all_labels = getRandomImageMasks(n=num_images, scale=0.20, shape=(512,512))
repainted_images = getRepaintedImages(imgs, masks)

100%|██████████| 50/50 [00:04<00:00, 10.51it/s]


In [10]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
adversarial_images = copy.deepcopy(imgs)
adversarial_results = model(adversarial_images)

repainted_results = model(repainted_images)

Using cache found in /homes/e34960/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-5 Python-3.9.16 torch-1.8.1+cu102 CUDA:0 (GeForce RTX 2080 Ti, 11019MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
for i in range(num_images):
    fig, axes = plt.subplots(1, 3, figsize=(12, 20))

    axes[0].imshow(imgs[i])
    axes[0].axis('off')
    axes[0].set_title('GT Bounding Boxes')

    for j in range(all_bboxes[i].shape[0]):
        if all_labels[i][j] != -10:
            rectangle = patches.Rectangle((all_bboxes[i][j][1], all_bboxes[i][j][0]), all_bboxes[i][j][3], all_bboxes[i][j][2], linewidth=1, edgecolor='r', facecolor='none')
            # cv2.putText(imgs[i], text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            axes[0].add_patch(rectangle)
            axes[0].text(all_bboxes[i][j][1], all_bboxes[i][j][0], annotations_coco_91[all_labels[i][j]], fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')
    

    adversarial_bboxes = (adversarial_results.pandas().xyxy[i]).to_numpy()
    adversarial_bboxes[:, 2] = (adversarial_bboxes[:, 2] - adversarial_bboxes[:, 0]).astype(int)
    adversarial_bboxes[:, 3] = (adversarial_bboxes[:, 3] - adversarial_bboxes[:, 1]).astype(int)
    adversarial_bboxes[:, 0] = (adversarial_bboxes[:, 0]).astype(int)
    adversarial_bboxes[:, 1] = (adversarial_bboxes[:, 1]).astype(int)

    axes[1].imshow(imgs[i])
    axes[1].axis('off')
    axes[1].set_title('Adversarial Bounding Boxes')
    for j in range(adversarial_bboxes.shape[0]):
        rectangle = patches.Rectangle((adversarial_bboxes[j][0], adversarial_bboxes[j][1]), adversarial_bboxes[j][2], adversarial_bboxes[j][3], linewidth=1, edgecolor='r', facecolor='none')
        axes[1].add_patch(rectangle)
        axes[1].text(adversarial_bboxes[j][0],  adversarial_bboxes[j][1]-10, "{}:{:.2f}".format(adversarial_bboxes[j][6], adversarial_bboxes[j][4]) , fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')

    # print(imgs[i].shape)
    # print(masks[i].shape)
    repainted_bboxes = (repainted_results.pandas().xyxy[i]).to_numpy()
    repainted_bboxes[:, 2] = (repainted_bboxes[:, 2] - repainted_bboxes[:, 0]).astype(int)
    repainted_bboxes[:, 3] = (repainted_bboxes[:, 3] - repainted_bboxes[:, 1]).astype(int)
    repainted_bboxes[:, 0] = (repainted_bboxes[:, 0]).astype(int)
    repainted_bboxes[:, 1] = (repainted_bboxes[:, 1]).astype(int)

    axes[2].imshow(repainted_images[i])
    axes[2].axis('off')
    axes[2].set_title('Repainted Bounding Boxes')
    for j in range(repainted_bboxes.shape[0]):
        rectangle = patches.Rectangle((repainted_bboxes[j][0], repainted_bboxes[j][1]), repainted_bboxes[j][2], repainted_bboxes[j][3], linewidth=1, edgecolor='r', facecolor='none')
        axes[2].add_patch(rectangle)
        axes[2].text(repainted_bboxes[j][0],  repainted_bboxes[j][1]-10, "{}:{:.2f}".format(repainted_bboxes[j][6], repainted_bboxes[j][4]) , fontsize=8, bbox=dict(facecolor='black', alpha=0.8, pad=1), color='white')


    plt.tight_layout()
    plt.show()
    display(fig)
    plt.close()